In [ ]:
# This cell is for installing the COCO API, which helps read the annotation files.
# It's typically pre-installed on Kaggle, but this ensures it's available.
!pip install pycocotools --quiet

import os
import json
import random
import numpy as np
from pycocotools.coco import COCO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

print("Libraries imported successfully.")

In [ ]:
# Define the paths to the dataset
# This path is standard for this dataset on Kaggle
BASE_PATH = '/kaggle/input/coco-2017-dataset/coco2017/'
IMG_PATH = os.path.join(BASE_PATH, 'train2017')
ANN_PATH = os.path.join(BASE_PATH, 'annotations', 'instances_train2017.json')

# Initialize the COCO API for instance annotations
print("Loading annotations... This may take a moment.")
coco = COCO(ANN_PATH)
print("Annotations loaded.")

In [ ]:
# Get all image IDs from the training set
all_img_ids = coco.getImgIds()
print(f"Total images in the training set: {len(all_img_ids)}")

# Collect the first 20,000 image IDs
image_ids_20k = all_img_ids[:20000]

print(f"Collected {len(image_ids_20k)} image IDs to work with.")

In [ ]:
# Select 50 images from our 20k list for visualization
# We'll take the first 50
images_to_visualize = image_ids_20k[:50]

print(f"Starting visualization for {len(images_to_visualize)} images...")

for img_id in images_to_visualize:
    # 1. Load Image Info
    img_info = coco.loadImgs([img_id])[0]
    img_path = os.path.join(IMG_PATH, img_info['file_name'])
    
    # 2. Load Original Image (as RGB)
    original_image = Image.open(img_path).convert('RGB')
    
    # 3. Get all annotations for this image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    annotations = coco.loadAnns(ann_ids)
    
    # 4. Create the segmentation mask using numpy and coco.annToMask
    # This is more robust as it handles both polygon and RLE formats.
    
    # Create a blank numpy array for the combined mask
    combined_mask_np = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
    
    # Iterate over all annotations (all subjects) for this image
    for ann in annotations:
        # coco.annToMask returns a binary 0/1 numpy array
        # This function correctly handles both RLE and polygon formats
        single_ann_mask_np = coco.annToMask(ann)
        
        # Combine masks using logical OR
        # This adds the current object's mask to the combined mask
        combined_mask_np = np.logical_or(combined_mask_np, single_ann_mask_np).astype(np.uint8)

    # Convert the final combined numpy mask (0/1) to a PIL Image (0/255)
    # We multiply by 255 to make the mask visible (white) for PIL
    mask_pil = Image.fromarray(combined_mask_np * 255, 'L')
    
    # 5. Create the final "Subject Isolated" image
    # This creates a black RGB background
    black_bg = Image.new('RGB', original_image.size, (0, 0, 0))
    
    # Use the PIL mask to composite the images
    # Where mask_pil is 255 (white), pixels from original_image are used.
    # Where mask_pil is 0 (black), pixels from black_bg are used.
    masked_image = Image.composite(original_image, black_bg, mask_pil)
    
    # 6. Visualize side-by-side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    
    # Original Image
    ax1.imshow(original_image)
    ax1.set_title(f"Original Image: {img_info['file_name']}")
    ax1.axis('off')
    
    # Masked Image (Subject Isolated)
    ax2.imshow(masked_image)
    ax2.set_title("Masked Image (Subject Isolated)")
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

print("Visualization complete.")